In [1]:
!pip install transformers datasets evaluate accelerate scikit-learn -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [2]:
import os
import torch
import numpy as np
import evaluate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset

os.environ["WANDB_DISABLED"] = "true"

# 1. LOAD DATASET
dataset = load_dataset("google-research-datasets/go_emotions", "simplified")


label_list = dataset["train"].features["labels"].feature.names
num_labels = len(label_list)

id2label = {idx: label for idx, label in enumerate(label_list)}
label2id = {label: idx for idx, label in enumerate(label_list)}

print(f"Total Labels: {num_labels}")

# 2. TOKENIZER & PREPROCESSING
MODEL_CKPT = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_CKPT)

def preprocess_function(examples):

    tokenized_inputs = tokenizer(examples["text"], truncation=True, padding=True, max_length=128)


    labels_matrix = np.zeros((len(examples["labels"]), num_labels), dtype=np.float32)
    for idx, label_indices in enumerate(examples["labels"]):
        labels_matrix[idx, label_indices] = 1.0

    tokenized_inputs["labels_new"] = labels_matrix.tolist()
    return tokenized_inputs

print("Sedang memproses data...")
tokenized_ds = dataset.map(preprocess_function, batched=True)


tokenized_ds = tokenized_ds.remove_columns(['labels', 'text', 'id'])


tokenized_ds = tokenized_ds.rename_column("labels_new", "labels")

tokenized_ds.set_format("torch")


print("\n[DEBUG] Cek Tipe Data Label:")
print(f"Tipe data: {tokenized_ds['train'][0]['labels'].dtype}")


# 3. METRIC
f1_metric = evaluate.load("f1")
acc_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))


    y_pred = np.zeros(probs.shape)
    y_pred[probs >= 0.5] = 1

    y_true_flat = labels.flatten()
    y_pred_flat = y_pred.flatten()

    f1 = f1_metric.compute(predictions=y_pred_flat, references=y_true_flat, average="micro")
    acc = acc_metric.compute(predictions=y_pred_flat, references=y_true_flat)

    return {"accuracy": acc["accuracy"], "f1": f1["f1"]}

# 4. TRAINING
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CKPT,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification"
)

args = TrainingArguments(
    output_dir="./go_emotions_final",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("Mulai Training...")
trainer.train()

# Simpan
trainer.save_model("./final_go_emotions_sukses")
print("Selesai!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

simplified/train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

simplified/validation-00000-of-00001.par(…):   0%|          | 0.00/350k [00:00<?, ?B/s]

simplified/test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2452837187.py:104: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Mulai Training...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.114400,0.094946,0.967662,0.967662
2,0.088600,0.086810,0.969532,0.969532
3,0.080400,0.085886,0.969683,0.969683


Selesai!
